In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q google-generativeai
!pip install PyPDF2
!pip install pdf2image
!pip install pdfminer.six
!pip install langchain
!pip install chromadb
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import spacy
from pdfminer.high_level import extract_text
import pprint
import google.generativeai as palm
import pandas as pd
from PyPDF2 import PdfReader
import numpy as np
import textwrap
import pdf2image
import os
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
import spacy
from pdfminer.high_level import extract_text
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GOOGLE_API_KEY"]='...'
llm = GooglePalm()
llm.temperature=0
folder_path = "/content/drive/MyDrive/WWF x DSS/pdfs_for_dss_cleaned"

In [ ]:
pdf_useful=pd.read_excel("PDFs useful.xlsx")

In [ ]:
pdf_useful_list=pdf_useful["PDF Name"]

In [ ]:
all_files = os.listdir(folder_path)
pdf_files = [file for file in all_files if file.endswith('.pdf')]

In [ ]:
# List to hold the dictionary for creating DataFrame later
data_for_df = []

In [ ]:
#Only use useful PDFs
filtered_pdf_files = [str(file).strip().lower() for file in pdf_files if str(file).strip().lower() in [str(name).strip().lower() for name in pdf_useful_list]]

In [ ]:
# Load SpaCy's NER model
nlp = spacy.load("en_core_web_sm")

# Function to extract locations using SpaCy
def extract_geolocations(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "GPE"]

# Assuming 'folder_path' is defined and 'pdf_names' is a list of PDF filenames

# Iterate over PDF filenames to extract locations
for pdf_name in filtered_pdf_files:
    pdf_path = os.path.join(folder_path, pdf_name)

    # Extract text from PDF
    text = extract_text(pdf_path)

    # Extract locations from text
    locations = extract_geolocations(text)

    # Append the results to our list
    data_for_df.append({"PDF Name": pdf_name, "Locations": locations})

In [ ]:
# Create a DataFrame
df_locations = pd.DataFrame(data_for_df)

# Show the DataFrame
df_locations

,PDF Name,Locations
0,941417b4-e2a6-4337-b3e9-7bb73a02ce56.pdf,"[Berkeley, Los Angeles, Washington, DC, Chile,..."
1,b524696c-d9a8-4905-98e3-e5ed4630721f.pdf,"[Kindle Fire, Kindle Fire, Kindle Fire, Kindle..."
2,b9b0aa20-292b-428f-90d3-f6c08cd5ae75.pdf,"[Nepal, Nepal, Himalaya, the United States \nA..."
3,a70e1021-627f-4a9d-86b1-5704a2a0891a.pdf,"[the United \nStates, US, Mozambique, TRILLION..."
4,acc1e526-4370-4134-97fe-e2951e314192.pdf,"[UK, Argentina, Argentina, Argentina, Argentin..."
...,...,...
196,746ff4fd-ff32-478a-a8f6-62ee19a755a6.pdf,"[Liman, North\t\r Sea\t\r Foundation, Leonar..."
197,4fa2b19f-eb82-4eeb-87a5-84fb732efedd.pdf,"[St. \n\nLouis, US, Washington, DC, California..."
198,4a8f8570-b9f5-43b7-b781-7c4a995bd218.pdf,"[US, US, US, NBS, Build, C., Ghile, Woodley, S..."
199,65898030-77a8-4065-b4cc-075ae9ee778b.pdf,"[London, Ginette Hemley, Gland, Switzerland, B..."


Identify only most relevant locations with PaLM from the list of locations extracted by spaCy

In [ ]:
import os
import re

# Define the special character removal function
def remove_special_characters(text):
    # This will replace anything that is not a letter, number or underscore with a space
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    # Remove multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

# Your existing code setup
text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
all_docs = []

In [ ]:
# Assuming combined_df is your existing DataFrame and folder_path is defined as your PDF directory

# Define a function to load, split, and create embeddings from a PDF document
def process_pdf(full_path):
    loader = PyPDFLoader(full_path)
    pages = loader.load_and_split()
    cleaned_pages = []
    for page in pages:
        if hasattr(page, 'page_content') and page.page_content:  # Make sure the attribute exists and is not None
            # Clean the page content
            page.page_content = remove_special_characters(page.page_content)
            cleaned_pages.append(page)  # Append the cleaned page to the new list
        else:
            # Handle pages with no content or no 'page_content' attribute as you see fit
            cleaned_pages.append(page)
    docs = text_splitter.split_documents(pages)
    embeddings = GooglePalmEmbeddings()
    return Chroma.from_documents(docs, embeddings)

In [ ]:
def filter_location_names(data):
    filtered_data = []

    for item in data:
        if '.' in item and '@' in item:
            # Exclude item language that PaLM can't take
            continue
        filtered_data.append(item)

    return filtered_data

In [ ]:
# Prepare the new column for the DataFrame
df_locations["Relevant Locations"] = None  # Initialize the new column

<ipython-input-72-26af4c980854>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_locations["Relevant Locations"] = None  # Initialize the new column


In [ ]:
for index, row in df_locations.iterrows():
    locations_list = row["Locations"]
    location_string = ", ".join(set(locations_list))
    full_path = os.path.join(folder_path, row["PDF Name"])
    db_new = process_pdf(full_path)

    qa = VectorDBQA.from_chain_type(llm=GooglePalm(), chain_type="stuff", vectorstore=db_new) #For computation simplicity, we use "stuff." Chain types like "map_reduce" could also work.

    query = f"From this list of geographical locations: {location_string}, identify which are most relevant based on the context of the text. Please respond with only the locations from this list, formatted as a comma-separated list."
    result = qa.run(query=query, temperature=0)
    df_locations.at[index, "Relevant Locations"] = result

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:253: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:253: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:253: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:253: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:253: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(
/usr/local/lib/pytho